In [1]:
import requests
import pandas as pd
import sqlite3


In [2]:
def get_lost_objects(gares, annees):
    results = []
    for gare in gares:
        for annee in annees:
            response = requests.get(f'https://a67b4b4d-cb00-45aa-9970-2c69b46cea8a@ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&lang=fr&rows=10000&sort=date&facet=date&facet=gc_obo_gare_origine_r_name&facet=gc_obo_type_c&refine.date={annee}&refine.gc_obo_gare_origine_r_name={gare}')
            results.append(response.json()["records"])
    return results



In [3]:
gares = [
    'Paris Bercy',
    'Paris Gare du Nord',
    'Paris Est',
    'Paris Gare de Lyon',
    'Paris Montparnasse',
    'Paris Saint-Lazare',
    'Paris Austerlitz'
    ]
annees = [annee for annee in range(2019,2023)]

lost_objects_list = get_lost_objects(gares, annees)

In [4]:
# connexion = sqlite3.connect('bdd.db')
# query = "SELECT id,nom FROM gare"
# df_gares = pd.read_sql(query, connexion)

def get_gare_id_by_name(name : str):

    referencies = {
        "lyon" : 1,
        "montparnasse" : 2,
        "nord" : 3,
        "lazare" : 4,
        "est" : 5,
        "bercy" : 6,
        'austerlitz' : 7,
    }
    for ref in referencies.keys():
        if ref in name.lower():
            return referencies[ref]
    return 404




In [5]:
objects_liste = []
for by_gareyear in lost_objects_list:
    for item in by_gareyear:
        item = item["fields"]
        objects_liste.append({"gare_id" : get_gare_id_by_name(item['gc_obo_gare_origine_r_name']), "type" : item['gc_obo_type_c'], "date" : item["date"][:10]})

In [6]:
len(objects_liste)

81632

In [7]:
count = 0 ######## verfication qu'aucun nom de gare ne correspond à aucun id 
for i in objects_liste:
    if i["gare_id"]==404:
        count+=1
count

0